# How Fast is the Bus Going on the Flower Street Bus-Only Lane? 

### Silver Line and Blue Line Express (910/950 and 860)
---
I am examining bus speeds (mph) and run times in different segments of the Flower Street Bus-Only Lane in Downtown Los Angeles. This bus-only lane runs from 3-7 pm. I am utilizing WIFI data from the bus. See my link to the [pre-processed WIFI data](http://lacmta-utils.metrocloudalliance.com/Download/Bus_Way_Data_2_ALL.txt). 

Here's a map of the bus-only lane: 

<img src="images/Picture1.png" width="600" height="200" >
    
##### Weeks (had to sample one week of the month, because of the 5 sec frequency):
* 5/6-5/10
* 6/17-6/21
* 7/15-7/19
* 8/12-8/16

###### Intersections (and dummy variables)
* Flower St/7th St,6483280,1840207,MT910 S,1
* Flower St/Olympic Blvd,6482060,1838634,MT910 S,2
* Flower St/Pico Blvd,6480834,1837068,MT910 S,3
* Flower St/Washington Blvd,6479617,1834807,MT910 S,4
* Flower St/23rd St,6478948,1833482,MT860-910 S,5
* Flower St/Adams Blvd,6478370,1832573,MT860-910 S,6


In [332]:
#import libraries 
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import re 
#import geopandas 

#check out the .txt file 
data = pd.read_csv('http://lacmta-utils.metrocloudalliance.com/Download/Bus_Way_Data_2_ALL.txt',
                  na_values = ['no info', '.'], skiprows = [1,2])

data.head(10)

,Trip#,Bus#,date,Fm_Time,To_Time,Del_secs,Direction,From_Stop#,To_Stop#,Trav_Distance
0,2001,3880,2019-05-06,06:03:48,06:05:23,95,N,3,4,1380
1,2001,3880,2019-05-06,06:05:23,06:06:18,55,N,4,5,2533
2,2001,3880,2019-05-06,06:06:18,06:07:13,55,N,5,6,2080
3,2001,3880,2019-05-06,06:07:13,06:08:03,50,N,6,7,2042
4,2002,5887,2019-05-06,08:34:40,08:35:30,50,N,2,3,318
5,2002,5887,2019-05-06,08:35:30,08:39:10,220,N,3,5,4116
6,2002,5887,2019-05-06,08:39:10,08:41:10,120,N,5,6,1920
7,2002,5887,2019-05-06,08:41:10,08:45:25,255,N,6,7,2116
8,2003,6058,2019-05-06,06:27:38,06:28:33,55,N,1,2,763
9,2003,6058,2019-05-06,06:28:33,06:29:13,40,N,2,3,326


In [333]:
#change values for Travel Distance and Time in Seconds to numeric 

data['Del_secs'] = pd.to_numeric(data['Del_secs'], errors='coerce')
data['Trav_Distance'] = pd.to_numeric(data['Trav_Distance'], errors='coerce')
data['date'] = pd.to_datetime(data['date'], errors='coerce')

In [334]:
data.Del_secs.dtype

dtype('float64')

In [335]:
data.Trav_Distance.dtype

dtype('float64')

In [336]:
data.date.dtype

dtype('<M8[ns]')

In [337]:
#taking a look at NAs 
data[4751:4753]

,Trip#,Bus#,date,Fm_Time,To_Time,Del_secs,Direction,From_Stop#,To_Stop#,Trav_Distance
4751,Trip#,Bus#,NaT,Fm_Time,To_Time,NaN,Direction,From_Stop#,To_Stop#,NaN
4752,==============================================...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [338]:
metro_data = data.dropna(subset=['Del_secs','Trav_Distance'])
metro_data = metro_data.reset_index(drop=True)
metro_data[4749:4755]

,Trip#,Bus#,date,Fm_Time,To_Time,Del_secs,Direction,From_Stop#,To_Stop#,Trav_Distance
4749,3070,9468,2019-08-16,07:38:44,07:41:09,145.0,N,5,6,2047.0
4750,3070,9468,2019-08-16,07:41:09,07:42:54,105.0,N,6,7,1987.0
4751,6001,3148,2019-07-15,06:14:25,06:15:05,40.0,N,2,3,253.0
4752,6001,3148,2019-07-15,06:15:05,06:15:40,35.0,N,3,4,1326.0
4753,6001,3148,2019-07-15,06:15:40,06:16:40,60.0,N,4,5,2841.0
4754,6001,3148,2019-07-15,06:16:40,06:17:25,45.0,N,5,6,1919.0


In [339]:
#check original data source
len(metro_data)

15489

In [340]:
#check your sample size - with NAs removed 
len(data)

15495

In [341]:
#one more check for good measure! 
print(metro_data.columns)

for col in metro_data.columns:
   print(np.dtype(metro_data[col].dtype))

Index(['Trip#', 'Bus#', 'date', 'Fm_Time', 'To_Time', 'Del_secs', 'Direction',
       'From_Stop#', 'To_Stop#', 'Trav_Distance'],
      dtype='object')
object
object
datetime64[ns]
object
object
float64
object
object
object
float64


In [342]:
# adding more columns with the miles per hour conversion
metro_data['hours'] = metro_data['Del_secs'] / 3600 
metro_data['miles'] = metro_data['Trav_Distance'] / 5280 
metro_data['mph'] = metro_data['miles'] / metro_data['hours']
metro_data['pm'] = metro_data['Fm_Time'].str.startswith('1')

In [343]:
metro_data.tail(10)

,Trip#,Bus#,date,Fm_Time,To_Time,Del_secs,Direction,From_Stop#,To_Stop#,Trav_Distance,hours,miles,mph,pm
15479,5441,9490,2019-08-16,16:37:54,16:49:19,685.0,N,6,7,2177.0,0.190278,0.412311,2.166888,True
15480,5442,9490,2019-08-16,17:28:09,17:30:24,135.0,S,1,2,2037.0,0.037500,0.385795,10.287879,True
15481,5442,9490,2019-08-16,17:30:24,17:33:09,165.0,S,2,3,1975.0,0.045833,0.374053,8.161157,True
15482,5442,9490,2019-08-16,17:33:09,17:35:49,160.0,S,3,4,2563.0,0.044444,0.485417,10.921875,True
15483,5442,9490,2019-08-16,17:35:49,17:37:59,130.0,S,4,5,1529.0,0.036111,0.289583,8.019231,True
15484,5442,9490,2019-08-16,17:37:59,17:39:14,75.0,S,5,6,1027.0,0.020833,0.194508,9.336364,True
15485,5443,9491,2019-08-16,19:45:44,19:46:19,35.0,N,3,4,1413.0,0.009722,0.267614,27.525974,True
15486,5443,9491,2019-08-16,19:46:19,19:48:34,135.0,N,4,5,2685.0,0.037500,0.508523,13.560606,True
15487,5443,9491,2019-08-16,19:48:34,19:49:24,50.0,N,5,6,2027.0,0.013889,0.383902,27.640909,True
15488,5443,9491,2019-08-16,19:49:24,19:52:04,160.0,N,6,7,1977.0,0.044444,0.374432,8.424716,True


In [344]:
#metro_data['date'] = metro_data['date'].dt.strftime('%m/%d/%Y')
#metro_data['date'].dtype

In [345]:
#metro_data['stopseq'] = str(metro_data['From_Stop#']) + str(metro_data['From_Stop#'])
#metro_data.head()

In [325]:
metro_data = (metro_data
            .query('mph < 50')
            .assign(prepost = lambda x: 'PrePilot' if (re.search(x['date'], '2019-05') is not None) else 'PostPilot')
    )

#metro_data[~(metro_data['mph'] > 50)]

#metro_data['Trip#'].count()
metro_data.head()

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [ ]:
#by_month = pd.pivot_table(victims, index = 'ACCIDENT_YEAR', 
                                   columns = 'VICTIM_ROLE', aggfunc='size')

In [ ]:
#filter by month -- Fig

#pre_flower = router_speed %>% 
  #filter(date == "May")

#post_flower = router_speed %>% 
  #filter(date != "May") 